## Running the Linear AA on the Beta Cells dataset

In [6]:
import torch
import numpy as np
import scanpy as sc
from src.methods.AABernoulli import Bernoulli_Archetypal_Analysis
from tqdm import tqdm
import torch
import numpy as np
from torchvision import datasets, transforms
from src.methods.AABernoulli import Bernoulli_Archetypal_Analysis
from tqdm import tqdm
import matplotlib.pyplot as plt 
from src.methods.AALS import AALS 
import sys, scprep, magic, scanpy, sklearn
from sklearn import decomposition




In [7]:

# 1. Load Data
print("Loading single-cell dataset...")
# Load the dataset (assuming 'data/beta_cells_hfd.h5ad' is accessible)
adata = sc.read_h5ad('data/beta_cells_hfd.h5ad')


# show size of dataset
print(f"Dataset shape: {adata.X.shape}")


Loading single-cell dataset...
Dataset shape: (3887, 16483)


In [8]:
# Dataset is too big, subsample for faster testing
adata = adata[:1000, :]
print(f"Subsampled dataset shape: {adata.X.shape}")

Subsampled dataset shape: (1000, 16483)


In [9]:
# 1. Standard Single-Cell Preprocessing (No Binarization)
# Convert to dense (if sparse) and apply log-transformation
X_dense = adata.X.toarray() if hasattr(adata.X, 'toarray') else adata.X.copy()
# Use scanpy's built-in log normalization for robustness
X_log_normalized = sc.pp.log1p(X_dense, copy=True)

### Running the Gaussian method on the cell data
- Used 4 as n_archetypes because that's what we used in the AANet implementation, for comparison.

In [10]:
# 2. MATCH PREPROCESSING TO AANET
# AAnet uses MAGIC + PCA. We must do the same for Linear AA.
print("Running MAGIC and PCA to match AAnet inputs...")
hfd_magic_op = magic.MAGIC(random_state=42, t=10)
hfd_magic = hfd_magic_op.fit_transform(adata.to_df())

hfd_pc_op = decomposition.PCA(n_components=20, random_state=42)
hfd_magic_pc = hfd_pc_op.fit_transform(hfd_magic)

# Normalize (Important: AAnet normalized by std dev, do the same)
hfd_magic_pc_norm = hfd_magic_pc / np.std(hfd_magic_pc[:, 0])

# 3. Prepare for Linear AA
# Convert the PCA output to tensor
X_tensor = torch.from_numpy(hfd_magic_pc_norm).float()

# Transpose: Shape becomes (Features x Cells) -> (20 x 1000)
X = X_tensor.t().double() 

n_archetypes = 4

# 4. Run AALS
C, A, X_recon, EV = AALS(X, n_archetypes)


Running MAGIC and PCA to match AAnet inputs...
Calculating MAGIC...
  Running MAGIC on 1000 cells and 16483 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...


/opt/anaconda3/envs/sae_env/lib/python3.11/site-packages/magic/magic.py:425: UserWarning: Input matrix contains unexpressed genes. Please remove them prior to running MAGIC.
  warnings.warn(


    Calculated PCA in 1.01 seconds.
    Calculating KNN search...
    Calculated KNN search in 0.05 seconds.
    Calculating affinities...
    Calculated affinities in 0.04 seconds.
  Calculated graph and diffusion operator in 1.13 seconds.
  Running MAGIC with `solver='exact'` on 16483-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.
  Calculating imputation...
  Calculated imputation in 0.51 seconds.
Calculated MAGIC in 1.70 seconds.


In [13]:
#print shapes
print(f"C shape: {C.shape}")
print(f"A shape: {A.shape}")
print(f"X_recon shape: {X_recon.shape}")
print(f"Explained Variance shape: {EV.shape}")

C shape: torch.Size([1000, 4])
A shape: torch.Size([4, 1000])


AttributeError: 'list' object has no attribute 'shape'

#### Save to avoid rerunning

In [ ]:
# Save the results for later visualization
torch.save({'C': C, 'A': A, 'X_recon': X_recon, 'EV': EV}, 'betacells_gaussian_aa_results_run5.pth')

In [ ]:
# Load 5 pth files (one for each run) and save a new one with 5 lists (one for each run)
C_list = []
A_list = []
X_recon_list = []
EV_list = []

for i in range(1, 6):
    results = torch.load(f'/Users/joaomata/Desktop/DTU/DeepLearning/ProjectDL/LinearAA/Python/gaussian_betacells_5runs/betacells_gaussian_aa_results_run{i}.pth')
    C_list.append(results['C'])
    A_list.append(results['A'])
    X_recon_list.append(results['X_recon'])
    EV_list.append(results['EV'])
    
torch.save({'C': C_list, 'A': A_list, 'X_recon': X_recon_list, 'EV': EV_list}, '/Users/joaomata/Desktop/DTU/DeepLearning/ProjectDL/LinearAA/Python/betacells_gaussian_aa_results_5runs.pth')

### Trying to visualize the results

In [ ]:
# Load the results for visualization
results = torch.load('betacells_gaussian_aa_results.pth')
C = results['C']
S = results['S']
L = results['L']


1. 🗺️ **What the UMAP Plot Shows** ($\mathbf{S}$ Matrix)

The UMAP plot visualizes the results of the $\mathbf{S}$ matrix (the coefficient matrix):
    - Cells (Points): The cells are arranged based on their gene expression similarity.
    - Archetype Assignment (Color): Each color represents the dominant archetype assignment for a cell, derived from the $\mathbf{S}$ matrix ($\text{Cell}_i$ is colored by the index of the highest coefficient in its column of $\mathbf{S}$).

Conclusion: The UMAP plot shows the clustering or grouping of your cells into the four archetypal populations.

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import torch

# --- 1. Ensure UMAP is Calculated on the Continuous Data ---
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.pca(adata) # Run PCA
# ADD THIS LINE: Calculate the k-nearest neighbors graph
sc.pp.neighbors(adata)
# Now UMAP can run successfully
sc.tl.umap(adata) # Run UMAP

# --- 2. Process S Matrix (Archetypal Assignment) ---

# S is expected to be a (n_archetypes x N_cells) tensor: (4, N_cells)
if isinstance(S, torch.Tensor):
    S_np = S.detach().cpu().numpy()
else:
    S_np = S.copy()

# Transpose S to be (N_cells x n_archetypes) so rows match the cells in adata.obs
S_cells = S_np.T

# Find the index (0, 1, 2, or 3) of the maximum coefficient for each cell.
# This index defines the cell's dominant archetype.
dominant_archetype_index = np.argmax(S_cells, axis=1)

# Add the assignment (plus 1 for labeling) back to the scanpy object
adata.obs['dominant_archetype'] = pd.Categorical(dominant_archetype_index + 1)

# --- 3. Plot the UMAP ---

sc.pl.umap(
    adata, 
    color='dominant_archetype', 
    title='UMAP Colored by Gaussian Archetype (k=4)',
    legend_loc='on data',
    frameon=False
)

In [ ]:
# --- 1. Correct Transposition and Gene Name Mismatch ---

# Check the input C matrix shape (4, 3887)
# We need (3887, 4) -> Transpose C_np
C_np = C_np.T

print(f"Corrected C_np shape: {C_np.shape}") # Should be (3887, 4)

# Get the gene names that correspond to the features used in the analysis.
# If you ran PCA on highly variable genes, those names are in adata.var_names
# filtered by the features actually used.
# Since C_np has 3887 rows, we must use 3887 gene names. 
# We'll assume the first 3887 genes in adata are the ones used. 
# You should verify this assumption based on your full data loading script.
gene_names = adata.var_names[:C_np.shape[0]].to_list() 

# 2. Re-create the DataFrame with Corrected Dimensions
C_df = pd.DataFrame(
    C_np, 
    index=gene_names, # Now correctly matched (3887 gene names)
    columns=[f'Archetype {i+1}' for i in range(C_np.shape[1])] # (4 Archetype columns)
)
print("DataFrame creation successful.")

# --- The rest of your visualization code continues here ---
# ... C_df['Max_Expr'] = C_df.max(axis=1) ...

# Find the maximum expression value across all 4 archetypes for each gene
C_df['Max_Expr'] = C_df.max(axis=1)

# Sort by Max_Expr and select the top 100 genes
C_top = C_df.sort_values(by='Max_Expr', ascending=False).head(100).drop(columns=['Max_Expr'])

# --- 3. Plot Heatmap ---

plt.figure(figsize=(6, 12))
sns.heatmap(
    C_top,
    cmap='magma', # High-contrast color map
    cbar_kws={'label': 'Gene Expression Level (C Value)'},
    yticklabels=True # Ensure gene names are labeled
)

plt.title('Top 100 Gaussian Archetype Gene Signatures')
plt.ylabel('Marker Genes')
plt.xlabel('Archetype Profile')
plt.yticks(rotation=0, fontsize=6)
plt.show()

In [ ]:
import pandas as pd # Ensure pandas is imported

# Set the number of top genes you want to see
N_TOP_GENES = 5

print(f"--- Top {N_TOP_GENES} Genes Defining Each of the Four Archetypes ---")
print("-" * 55)

# Iterate through each column (Archetype) in the DataFrame
for col in C_df.columns:
    # Sort the current archetype column in descending order
    top_genes = C_df[col].sort_values(ascending=False).head(N_TOP_GENES)
    
    print(f"\n🔬 {col}:")
    # Print the gene name (index) and its contribution value
    for gene, contribution in top_genes.items():
        print(f"  {gene:<20} {contribution:.4f}")
    
print("-" * 55)